In [1]:
import pandas as pd

In [2]:
df_en = pd.read_json('wiki_en.json')

In [3]:
df_ru = pd.read_json('wiki_ru.json')

Merge по изображениям

In [4]:
df_en_ru = df_en.merge(df_ru, on=['image_url'], how='left', indicator=True)
df_en_ru

,image_url,caption_description_x,caption_description_y,_merge
0,https://upload.wikimedia.org/wikipedia/commons...,"Nameplate of Gazeta Transilvaniei on June 14, ...",NaN,left_only
1,https://upload.wikimedia.org/wikipedia/commons...,A captive decoy,Подсадная птица,both
2,https://upload.wikimedia.org/wikipedia/commons...,The former Bus. 41 along Washington Street in...,NaN,left_only
3,https://upload.wikimedia.org/wikipedia/commons...,"Viktor Vekselberg, head of the Skolkovo Instit...",Президент Фонда «» Виктор Вексельберг,both
4,https://upload.wikimedia.org/wikipedia/commons...,Volunteer planting poppies,NaN,left_only
...,...,...,...,...
2349693,https://upload.wikimedia.org/wikipedia/commons...,Massachusetts Bay Transportation Authority 170...,NaN,left_only
2349694,https://upload.wikimedia.org/wikipedia/commons...,Fortified church,NaN,left_only
2349695,https://upload.wikimedia.org/wikipedia/commons...,Bisaldeo temple in 2016 with submerged courtyard,NaN,left_only
2349696,https://upload.wikimedia.org/wikipedia/commons...,Beyer in 2013,NaN,left_only


In [5]:
df_ru_en = df_ru.merge(df_en, on=['image_url'], how='left', indicator=True)
df_ru_en

,image_url,caption_description_x,caption_description_y,_merge
0,https://upload.wikimedia.org/wikipedia/commons...,Владислав Захаревич,NaN,left_only
1,https://upload.wikimedia.org/wikipedia/commons...,Зарисовка крупнейшего из Золотых ромбов курган...,Design of the larger of the Bush Barrow gold l...,both
2,https://upload.wikimedia.org/wikipedia/commons...,Трассовый моделизм. Прохождение моделями виража,NaN,left_only
3,https://upload.wikimedia.org/wikipedia/commons...,Спаривание,NaN,left_only
4,https://upload.wikimedia.org/wikipedia/commons...,Музей Дмитрия Бурылина,Industrial and art museum,both
...,...,...,...,...
449907,https://upload.wikimedia.org/wikipedia/commons...,Изображение мужской особи человека (самец вида...,NaN,left_only
449908,https://upload.wikimedia.org/wikipedia/commons...,Пасхальное украшение. ЖД станция в Норрчёпинге...,NaN,left_only
449909,https://upload.wikimedia.org/wikipedia/commons...,Вид на «Красный ». Осень 2017 г,NaN,left_only
449910,http://upload.wikimedia.org/wikipedia/commons/...,Максимилиан фон Родаковский во главе 13-го пол...,Rodakowski at the Battle of Custozza. (1908. O...,both


In [6]:
mask = (df_en_ru['_merge'] == 'left_only') #изображения, которые есть только в df_en
mask.value_counts()

True     2184123
False     165575
Name: _merge, dtype: int64

In [7]:
mask_ru_en = (df_ru_en['_merge'] == 'left_only') #изображения, которые есть только в df_ru
mask_ru_en.value_counts()

True     284337
False    165575
Name: _merge, dtype: int64

In [8]:
df_to_translate_ru_en = df_ru_en[mask_ru_en]

In [9]:
df_to_translate_ru_en.index = list(range(df_to_translate_ru_en.shape[0]))

In [10]:
size = df_to_translate_ru_en.shape[0]
size

284337

In [2]:
from itertools import chain

In [15]:
import requests
import re
def translate_req(q, sl, tl):
    proxies = {'http':  'socks5://127.0.0.1:9050',
               'https': 'socks5://127.0.0.1:9050'}
    url = 'http://translate.google.com/m'
    R = r'(?s)class="(?:t0|result-container)">(.*?)<'
    params = {'q' : q, 'sl': sl, 'tl': tl}
    try:
        request_get = requests.get(url, proxies=proxies, params=params).text
        return re.findall(R, request_get)
    except:
        return None

def translate_ru_en(i):
    result = []
    start = int(i * size // N)
    end = int((i + 1) * size // N) if i < N - 1 else size
    print(start, end)
    for index, row in df_to_translate_ru_en.iloc[start:end].iterrows():
        req = translate_req(q=row['caption_description_x'], sl='ru', tl='en')
        if req is not None:
            result.append((index, req))
    return result

In [17]:
from joblib import Parallel
from joblib import delayed
N = 20
size = df_to_translate_ru_en.shape[0]
data = Parallel(n_jobs=N, verbose=10)(delayed(translate_ru_en)(k) for k in list(range(N)))
result = list(chain(*data))

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed: 489.5min remaining: 4405.1min
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed: 497.7min remaining: 1493.2min
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed: 508.9min remaining: 763.3min
[Parallel(n_jobs=20)]: Done  11 out of  20 | elapsed: 517.8min remaining: 423.7min
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed: 532.4min remaining: 228.2min
[Parallel(n_jobs=20)]: Done  17 out of  20 | elapsed: 581.7min remaining: 102.6min
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed: 701.2min remaining:    0.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed: 701.2min finished


In [18]:
df_result = pd.DataFrame(result)
df_result = df_result.rename(columns={'caption_reference_description' : 'caption_description'})
df_result

,0,1
0,0,[Vladislav Zakharevich]
1,1,[Track modeling. Models passing a bend]
2,2,[Pairing]
3,3,[Wise Tsar Berendey. Sketch by Viktor Vasnetso...
4,4,[Coat of arms of Stahl von Holstein]
...,...,...
216910,284332,[Vadim Ilyich Baranov and Sergei Pavlovich Zal...
216911,284333,[Image of a male human (male of the species) s...
216912,284334,[Easter decoration. Railway station in Norrköp...
216913,284335,[View of &quot;Red&quot;. Autumn 2017]


In [19]:
json = df_result.to_json(orient="records")
with open("wiki_translate_ru_en.json", "w") as outfile: 
    outfile.write(json)

In [11]:
df_result = pd.read_json('wiki_translate_ru_en.json')

In [12]:
df_result.head()

,0,1
0,0,[Vladislav Zakharevich]
1,1,[Track modeling. Models passing a bend]
2,2,[Pairing]
3,3,[Wise Tsar Berendey. Sketch by Viktor Vasnetso...
4,4,[Coat of arms of Stahl von Holstein]


In [13]:
df_result = df_result.rename(columns={0: 'ind', 1: 'caption_description_en'})
df_result.columns

Index(['ind', 'caption_description_en'], dtype='object')

In [14]:
df_result['caption_description_en'] = df_result['caption_description_en'].apply(lambda x: str(x[0]) if (type(x) is list and len(x) > 0) else x)
df_result

,ind,caption_description_en
0,0,Vladislav Zakharevich
1,1,Track modeling. Models passing a bend
2,2,Pairing
3,3,Wise Tsar Berendey. Sketch by Viktor Vasnetsov...
4,4,Coat of arms of Stahl von Holstein
...,...,...
216910,284332,Vadim Ilyich Baranov and Sergei Pavlovich Zalygin
216911,284333,Image of a male human (male of the species) se...
216912,284334,Easter decoration. Railway station in Norrköpi...
216913,284335,View of &quot;Red&quot;. Autumn 2017


In [15]:
(df_result['caption_description_en'] == '').value_counts()

False    216844
True         71
Name: caption_description_en, dtype: int64

In [16]:
df_result = df_result[df_result['caption_description_en'] != '']

In [17]:
df_to_translate_ru_en.columns

Index(['image_url', 'caption_description_x', 'caption_description_y',
       '_merge'],
      dtype='object')

In [18]:
pd.options.mode.chained_assignment = None

In [19]:
df_to_translate_ru_en.loc[:, "ind"] = list(df_to_translate_ru_en.index.values)

In [20]:
df_translate_ru_en = pd.merge(df_to_translate_ru_en, df_result, left_on='ind', right_on='ind', how='left')

In [21]:
df_translate_ru_en = df_translate_ru_en[['image_url', 'caption_description_x', 'caption_description_en']]
df_translate_ru_en = df_translate_ru_en.rename(columns={'caption_description_x': 'caption_description_ru'})
df_translate_ru_en

,image_url,caption_description_ru,caption_description_en
0,https://upload.wikimedia.org/wikipedia/commons...,Владислав Захаревич,Vladislav Zakharevich
1,https://upload.wikimedia.org/wikipedia/commons...,Трассовый моделизм. Прохождение моделями виража,Track modeling. Models passing a bend
2,https://upload.wikimedia.org/wikipedia/commons...,Спаривание,Pairing
3,https://upload.wikimedia.org/wikipedia/commons...,Премудрый царь Берендей. Эскиз Виктора Васнецо...,Wise Tsar Berendey. Sketch by Viktor Vasnetsov...
4,https://upload.wikimedia.org/wikipedia/commons...,Герб Сталь-фон-Гольштейнов,Coat of arms of Stahl von Holstein
...,...,...,...
284332,https://upload.wikimedia.org/wikipedia/commons...,Вадим Ильич Баранов и Сергей Павлович Залыгин,Vadim Ilyich Baranov and Sergei Pavlovich Zalygin
284333,https://upload.wikimedia.org/wikipedia/commons...,Изображение мужской особи человека (самец вида...,Image of a male human (male of the species) se...
284334,https://upload.wikimedia.org/wikipedia/commons...,Пасхальное украшение. ЖД станция в Норрчёпинге...,Easter decoration. Railway station in Norrköpi...
284335,https://upload.wikimedia.org/wikipedia/commons...,Вид на «Красный ». Осень 2017 г,View of &quot;Red&quot;. Autumn 2017


In [22]:
df_ru_en_both = df_ru_en[df_ru_en['_merge'] == 'both'][['image_url', 'caption_description_x', 'caption_description_y']]
df_ru_en_both = df_ru_en_both.rename(columns={'caption_description_x': 'caption_description_ru', 'caption_description_y': 'caption_description_en'})

In [23]:
df_both_translate_ru_en = pd.concat([df_translate_ru_en, df_ru_en_both], ignore_index=True) # совпадающие + перевод ru -> en
df_both_translate_ru_en

,image_url,caption_description_ru,caption_description_en
0,https://upload.wikimedia.org/wikipedia/commons...,Владислав Захаревич,Vladislav Zakharevich
1,https://upload.wikimedia.org/wikipedia/commons...,Трассовый моделизм. Прохождение моделями виража,Track modeling. Models passing a bend
2,https://upload.wikimedia.org/wikipedia/commons...,Спаривание,Pairing
3,https://upload.wikimedia.org/wikipedia/commons...,Премудрый царь Берендей. Эскиз Виктора Васнецо...,Wise Tsar Berendey. Sketch by Viktor Vasnetsov...
4,https://upload.wikimedia.org/wikipedia/commons...,Герб Сталь-фон-Гольштейнов,Coat of arms of Stahl von Holstein
...,...,...,...
449907,https://upload.wikimedia.org/wikipedia/commons...,С Эдмондом ’Брайеном в одном из эпизодов сериа...,Edmond O'Brien and Yvonne Craig (1963
449908,https://upload.wikimedia.org/wikipedia/commons...,К. Е. Маковский. Запорожский казак (1884). Сум...,"Zaporozhian Cossack by Konstantin Makovsky, 1884"
449909,https://upload.wikimedia.org/wikipedia/commons...,Новогодняя Волга (северная окраина Ярославля,A completely frozen Volga River in Yaroslavl (...
449910,https://upload.wikimedia.org/wikipedia/commons...,Кафедральный собор св. Николая,The Uniate cathedral


In [27]:
json = df_both_translate_ru_en.to_json(orient="records")
with open("wiki_df_both_translate_ru_en.json", "w") as outfile: 
    outfile.write(json)

In [30]:
df_to_translate_en_ru = df_en_ru[mask]

In [31]:
df_to_translate_en_ru.index = list(range(df_to_translate_en_ru.shape[0]))

In [25]:
import requests
import re
def translate_req(q, sl, tl):
    proxies = {'http':  'socks5://127.0.0.1:9050',
               'https': 'socks5://127.0.0.1:9050'}
    url = 'http://translate.google.com/m'
    R = r'(?s)class="(?:t0|result-container)">(.*?)<'
    params = {'q' : q, 'sl': sl, 'tl': tl}
    try:
        request_get = requests.get(url, proxies=proxies, params=params).text
        return re.findall(R, request_get)
    except:
        return None

def translate_en_ru(i):
    result = []
    size = df_i.shape[0]
    start = int(i * size // N)
    end = int((i + 1) * size // N) if i < N - 1 else size
    print(start, end)
    for index, row in df_i.iloc[start:end].iterrows():
        req = translate_req(q=row['caption_description_x'], sl='en', tl='ru')
        if req is not None:
            result.append((index, req))
    return result

In [15]:
from joblib import Parallel
from joblib import delayed
N = 20
result_en_ru = []
size = df_to_translate_en_ru.shape[0]
for i in range(size // 200000):
    df_i = df_to_translate_en_ru.iloc[i * 200000: (i + 1) * 200000 if (i + 1) * 200000 < size else size]
    data_en_ru = Parallel(n_jobs=N, verbose=10)(delayed(translate_en_ru)(k) for k in list(range(N)))
    result_en_ru = result_en_ru + list(chain(*data_en_ru))

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed: 471.8min remaining: 4245.9min
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed: 472.1min remaining: 1416.3min
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed: 472.6min remaining: 708.9min
[Parallel(n_jobs=20)]: Done  11 out of  20 | elapsed: 473.3min remaining: 387.2min
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed: 473.9min remaining: 203.1min
[Parallel(n_jobs=20)]: Done  17 out of  20 | elapsed: 475.1min remaining: 83.8min
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed: 476.6min remaining:    0.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed: 476.6min finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed: 433.0min remaining: 3896.9min
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed: 433.7min remaining: 1301.0min
[Parallel(n_jobs=20)]: Do

In [16]:
df_result_en_ru = pd.DataFrame(result_en_ru)

In [17]:
json = df_result_en_ru.to_json(orient="records")
with open("wiki_translate_en_ru.json", "w") as outfile: 
    outfile.write(json)

In [24]:
df_result_en_ru = pd.read_json('wiki_translate_en_ru.json')

In [25]:
df_result_en_ru = df_result_en_ru.rename(columns={0: 'ind', 1: 'caption_description_ru'})

In [26]:
df_result_en_ru['caption_description_ru'] = df_result_en_ru['caption_description_ru'].apply(lambda x: str(x[0]) if (type(x) is list and len(x) > 0) else x)
df_result_en_ru

,ind,caption_description_ru
0,0,Табличка Gazeta Transilvaniei от 14 июня 1936 ...
1,1,Бывший Автобус. 41 по Вашингтон-стрит в центре...
2,2,Волонтеры сажают маки.
3,3,Схема электромагнитного насоса
4,4,"Скульптура Антонио Масео работы Бони, 1916 год."
...,...,...
1999235,1999995,Фасад вокзала Грумбридж
1999236,1999996,Титчмарш на Всемирной живой выставке садоводов...
1999237,1999997,В 1995 году мост межштатной автомагистрали 80 ...
1999238,1999998,Печать Амори как граф Глостер


In [27]:
(df_result_en_ru['caption_description_ru'] == '').value_counts()

False    1999132
True         108
Name: caption_description_ru, dtype: int64

In [28]:
df_result_en_ru = df_result_en_ru[df_result_en_ru['caption_description_ru'] != '']

In [32]:
df_to_translate_en_ru.loc[:, "ind"] = list(df_to_translate_en_ru.index.values)

In [33]:
df_translate_en_ru = pd.merge(df_to_translate_en_ru, df_result_en_ru, left_on='ind', right_on='ind', how='left')

In [34]:
df_translate_en_ru

,image_url,caption_description_x,caption_description_y,_merge,ind,caption_description_ru
0,https://upload.wikimedia.org/wikipedia/commons...,"Nameplate of Gazeta Transilvaniei on June 14, ...",NaN,left_only,0,Табличка Gazeta Transilvaniei от 14 июня 1936 ...
1,https://upload.wikimedia.org/wikipedia/commons...,The former Bus. 41 along Washington Street in...,NaN,left_only,1,Бывший Автобус. 41 по Вашингтон-стрит в центре...
2,https://upload.wikimedia.org/wikipedia/commons...,Volunteer planting poppies,NaN,left_only,2,Волонтеры сажают маки.
3,https://upload.wikimedia.org/wikipedia/commons...,Schematic of Electromagnetic Pump,NaN,left_only,3,Схема электромагнитного насоса
4,https://upload.wikimedia.org/wikipedia/commons...,"Boni's sculpture of Antonio Maceo, 1916",NaN,left_only,4,"Скульптура Антонио Масео работы Бони, 1916 год."
...,...,...,...,...,...,...
2184118,https://upload.wikimedia.org/wikipedia/commons...,Massachusetts Bay Transportation Authority 170...,NaN,left_only,2184118,NaN
2184119,https://upload.wikimedia.org/wikipedia/commons...,Fortified church,NaN,left_only,2184119,NaN
2184120,https://upload.wikimedia.org/wikipedia/commons...,Bisaldeo temple in 2016 with submerged courtyard,NaN,left_only,2184120,NaN
2184121,https://upload.wikimedia.org/wikipedia/commons...,Beyer in 2013,NaN,left_only,2184121,NaN


In [35]:
df_translate_en_ru = df_translate_en_ru[['image_url', 'caption_description_x', 'caption_description_ru']]
df_translate_en_ru = df_translate_en_ru.rename(columns={'caption_description_x': 'caption_description_en'})
df_translate_en_ru

,image_url,caption_description_en,caption_description_ru
0,https://upload.wikimedia.org/wikipedia/commons...,"Nameplate of Gazeta Transilvaniei on June 14, ...",Табличка Gazeta Transilvaniei от 14 июня 1936 ...
1,https://upload.wikimedia.org/wikipedia/commons...,The former Bus. 41 along Washington Street in...,Бывший Автобус. 41 по Вашингтон-стрит в центре...
2,https://upload.wikimedia.org/wikipedia/commons...,Volunteer planting poppies,Волонтеры сажают маки.
3,https://upload.wikimedia.org/wikipedia/commons...,Schematic of Electromagnetic Pump,Схема электромагнитного насоса
4,https://upload.wikimedia.org/wikipedia/commons...,"Boni's sculpture of Antonio Maceo, 1916","Скульптура Антонио Масео работы Бони, 1916 год."
...,...,...,...
2184118,https://upload.wikimedia.org/wikipedia/commons...,Massachusetts Bay Transportation Authority 170...,NaN
2184119,https://upload.wikimedia.org/wikipedia/commons...,Fortified church,NaN
2184120,https://upload.wikimedia.org/wikipedia/commons...,Bisaldeo temple in 2016 with submerged courtyard,NaN
2184121,https://upload.wikimedia.org/wikipedia/commons...,Beyer in 2013,NaN


In [36]:
df_all = pd.concat([df_translate_en_ru, df_ru_en_both], ignore_index=True) # совпадающие + перевод ru -> en + перевод en -> ru
df_all

,image_url,caption_description_en,caption_description_ru
0,https://upload.wikimedia.org/wikipedia/commons...,"Nameplate of Gazeta Transilvaniei on June 14, ...",Табличка Gazeta Transilvaniei от 14 июня 1936 ...
1,https://upload.wikimedia.org/wikipedia/commons...,The former Bus. 41 along Washington Street in...,Бывший Автобус. 41 по Вашингтон-стрит в центре...
2,https://upload.wikimedia.org/wikipedia/commons...,Volunteer planting poppies,Волонтеры сажают маки.
3,https://upload.wikimedia.org/wikipedia/commons...,Schematic of Electromagnetic Pump,Схема электромагнитного насоса
4,https://upload.wikimedia.org/wikipedia/commons...,"Boni's sculpture of Antonio Maceo, 1916","Скульптура Антонио Масео работы Бони, 1916 год."
...,...,...,...
2349693,https://upload.wikimedia.org/wikipedia/commons...,Edmond O'Brien and Yvonne Craig (1963,С Эдмондом ’Брайеном в одном из эпизодов сериа...
2349694,https://upload.wikimedia.org/wikipedia/commons...,"Zaporozhian Cossack by Konstantin Makovsky, 1884",К. Е. Маковский. Запорожский казак (1884). Сум...
2349695,https://upload.wikimedia.org/wikipedia/commons...,A completely frozen Volga River in Yaroslavl (...,Новогодняя Волга (северная окраина Ярославля
2349696,https://upload.wikimedia.org/wikipedia/commons...,The Uniate cathedral,Кафедральный собор св. Николая


In [ ]:
json = df_all.to_json(orient="records")
with open("wiki_all_en_ru.json", "w") as outfile: 
    outfile.write(json)